In [1]:
import numpy as np
import itertools
import pandas as pd
from textblob import TextBlob
import re

In [2]:
nick = {}
with open('name_list.txt', 'rb') as f:
    for line in f:
        if line.strip().partition("\t")[-1] != '':
            nick[line.strip().partition("\t")[0]] = line.strip().partition("\t")[0].split(' ')+line.strip().partition("\t")[-1][1:-1].split(', ')
        else:
            nick[line.strip().partition("\t")[0]] = line.strip().partition("\t")[0].split(' ')
f.close()

In [3]:
names_v1 = []
for x in nick.keys():
    with open('sentences2.txt', 'rb') as f:
        for line in f:
            if x.split(' ')[0].lower() in re.findall(r"[\w']+", line.lower()):
                names_v1.append(x)
            if nick[x][2:] != []:
                for name in nick[x][2:]:
                    if set(name.lower().split(' ')).intersection(re.findall(r"[\w']+", line.lower())) ==  set(name.lower().split(' ')):
                        names_v1.append(x)

In [4]:
allnames = sorted(list(set(names_v1)))

In [5]:
for x in allnames:
    if len(x.split(' ')) == 1:
        allnames.remove(x)
    if (len(x.split(' ')) == 3 and x.split(' ')[-1].lower() == 'potter'):
        allnames.remove(x)

In [6]:
# need to run it again
for x in allnames:
    if len(x.split(' ')) == 1:
        allnames.remove(x)
    if (len(x.split(' ')) == 3 and x.split(' ')[-1].lower() == 'potter'):
        allnames.remove(x)

In [7]:
names_set = []
for a,b in itertools.combinations(allnames,2):
    names_set.append([a,b])

In [8]:
ep = {}
for x in allnames:
    ep[x] = nick[x]
    
#import pickle
#with open("nick_ep1", "wb") as f:
#    pickle.dump(ep1, f)

In [9]:
m1 = pd.DataFrame(np.zeros(shape=(len(allnames),len(allnames))),index = [range(len(allnames))], columns = [range(len(allnames))])
m1.index = allnames
m1.columns = allnames

m2 = pd.DataFrame(np.zeros(shape=(len(allnames),len(allnames))),index = [range(len(allnames))], columns = [range(len(allnames))])
m2.index = allnames
m2.columns = allnames

In [10]:
import time
start_time = time.time()
for x,y in names_set:
    with open('sentences2.txt', 'rb') as f:
        temp1 = 0
        temp2 = 0
        for line in f:
            for name1, name2 in itertools.product(ep[x], ep[y]):
                if name1 in ['Mrs Weasley', 'The Grey'] or name2 in ['Mrs Weasley', 'The Grey']:
                    if set(name1.lower().split(' ')).intersection(re.findall(r"[\w']+", line.lower())) ==  set(name1.lower().split(' ')) and set(name2.lower().split(' ')).intersection(re.findall(r"[\w']+", line.lower())) ==  set(name2.lower().split(' ')):
                        textblob = TextBlob(line.decode('utf8').strip())
                        temp1 += textblob.sentiment.polarity
                        temp2 += textblob.sentiment.subjectivity 
                else:
                    if name1.lower() in re.findall(r"[\w']+", line.lower()) and name2.lower() in re.findall(r"[\w']+", line.lower()):
                        textblob = TextBlob(line.decode('utf8').strip())
                        temp1 += textblob.sentiment.polarity
                        temp2 += textblob.sentiment.subjectivity 
                          
    m1.loc[x,y] = np.copy(temp1)
    m2.loc[x,y] = np.copy(temp2)
    m1.loc[y,x] = np.copy(temp1)
    m2.loc[y,x] = np.copy(temp2)

In [11]:
time.time()-start_time

811.8221271038055

In [12]:
family_matrix = pd.read_csv("book2_co_occurrence.csv", index_col=0)

In [13]:
for r in family_matrix.index:
    for c in family_matrix.columns:
        if family_matrix.loc[r,c] != 0:
            m1.loc[r,c] = m1.loc[r,c]/family_matrix.loc[r,c]

In [14]:
for r in family_matrix.index:
    for c in family_matrix.columns:
        if family_matrix.loc[r,c] != 0:
            m2.loc[r,c] = m2.loc[r,c]/family_matrix.loc[r,c]

In [16]:
m1.to_csv("book2_polarity.csv")

In [17]:
m2.to_csv("book2_subjectivity.csv")